In [5]:
import cv2
import numpy as np

def rescale_image(image, target_width):
    height, width = image.shape[:2]
    scale_factor = target_width / width
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def convert_hue(hue):
    return cv2.applyColorMap((hue*(255/179)).astype('uint8'), cv2.COLORMAP_HSV)

def count_circles(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # custom params
    circles = cv2.HoughCircles(im ,cv2.HOUGH_GRADIENT, 1, 50, param1=100,param2=20,minRadius=10, maxRadius=50)
    result = 0
    if circles is not None:
        circles = np.uint16(np.around(circles))
        result = len(circles[0,:])

    return result
    
def count_coins(file):
    img = cv2.imread(file)
    img = rescale_image(img, 640)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    mask_yellow = cv2.inRange(img_hsv, (20, 100, 0), (50, 255, 255))
    mask_yellow = cv2.medianBlur(mask_yellow, 5)
    mask_blue = cv2.inRange(img_hsv, (90, 100, 100), (120, 255, 255))
    mask_blue = cv2.medianBlur(mask_blue, 5)

    im_yellow = cv2.bitwise_and(img, img, mask=mask_yellow)
    yellow = count_circles(im_yellow)
    
    im_blue = cv2.bitwise_and(img, img, mask=mask_blue)
    blue = count_circles(im_blue)

    cv2.putText(img, f'y: {yellow} b: {blue}', (50, 50), cv2.FONT_HERSHEY_PLAIN, 5, (0,0,0), 2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return yellow, blue
    

In [6]:
for i in range(1, 11):
    print(i, ":", count_coins(f"coin{i}.jpg"))

1 : (5, 8)
2 : (6, 3)
3 : (2, 4)
4 : (2, 4)
5 : (1, 7)
6 : (3, 5)
7 : (4, 3)
8 : (5, 5)
9 : (2, 6)
10 : (4, 2)
